# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [13]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [14]:
# Creating a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [15]:
# Setting the keyspace 

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Creating the first table

In [42]:
## Query 1:  Print artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# A table specific to the query has to be created 
# Here the PRIMARY KEY is chosen to be ((session_id), item_in_session) being session_id the partition key
# We are querying on these two attributes and session_id will be the better choice as PARTITION KEY as it will be hashed to find the cluster
# Since we need to query on item_in_session, it is chosen to be a clustering column and is added in the PRIMARY KEY 

drop = "DROP TABLE IF EXISTS music_library "
create = "CREATE TABLE IF NOT EXISTS music_library "
create = create + "(session_id int, item_in_session int, artist_name varchar, song_title varchar, song_duration float  \
                    ,PRIMARY KEY ((session_id), item_in_session))                                                      ;"

try:
    session.execute(drop)
    session.execute(create)
except Exception as e:
    print(e)

                    

#### Inserting the data into first table

In [43]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Inserting values into the table
        query = "INSERT INTO music_library (session_id, item_in_session, artist_name, song_title, song_duration)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assigning the column elements with each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into first table

In [45]:
## Executing the select query to verify if the data was entered into the table

query = "SELECT artist_name, song_title, song_duration \
         FROM music_library                            \
         WHERE SESSION_ID = 338                        \
             AND ITEM_IN_SESSION = 4                   ;"               
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_duration)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Creating the second table

In [49]:
## Select name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# A table specific to the query has to be created 
# Here the PRIMARY KEY is chosen to be ((user_id, session_id), item_in_session) being (user_id, sessionId) the partition key
# We are querying on these two attributes and we want the results to be sorted in order of item_in_session
# If only user_id was used as PARTITION KEY, then there exists a high chance of data aggregating in a single node (depends on data)
# Hence using (user_id, session_id) as PARTITION KEY, it is more likely to be distributed across nodes as session_id changes periodically
# Since the output needs to be sorted in item_in_session, it is chosen to be a clustering column and is added in the PRIMARY KEY  

drop = "DROP TABLE IF EXISTS artist_library "
create = "CREATE TABLE IF NOT EXISTS artist_library "
create = create + "(user_id int, session_id int, item_in_session int, artist_name varchar, song_title text   \
                    ,first_name varchar, last_name varchar                                                   \
                    ,PRIMARY KEY ((user_id, session_id), item_in_session))                                   ;"

try:
    session.execute(drop)
    session.execute(create)
except Exception as e:
    print(e)                  


                    

#### Inserting the data in the second table

In [51]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Inserting values into the table
        query = "INSERT INTO artist_library (user_id, session_id, item_in_session, artist_name    \
                                             ,song_title, first_name, last_name)                  "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assigning the column elements with each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into second table

In [52]:
## Executing the select query to verify if the data was entered into the table

query = "SELECT artist_name, song_title, first_name, last_name \
         FROM artist_library                                   \
         WHERE user_id = 10                                    \
             AND session_id = 182                              ;"               
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Creating the third table

In [66]:
## Select every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# A table specific to the query has to be created 
# Here the PRIMARY KEY is chosen to be ((song_title), user_id) being song_title the partition key
# We are querying on the song_title and hence it should be the PARTITION KEY
# If only song_title was as PRIMARY KEY, since Cassandra does not allow duplicate entries, the entries will be overwritten
# Hence using (song_title, user_id) as PRIMARY KEY, we can overcome this issue without violating the query constraints


drop = "DROP TABLE IF EXISTS play_history "
create = "CREATE TABLE IF NOT EXISTS play_history "
create = create + "(song_title text, user_id int, first_name varchar, last_name varchar    \
                  ,PRIMARY KEY ((song_title), user_id))                                    ;"

try:
    session.execute(drop)
    session.execute(create)
except Exception as exep:
    print(exep)


#### Inserting the data in the third table

In [67]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        # Insert into the table
        query = "INSERT INTO play_history (song_title, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        
        # Properly assign the values from CSV to the table attributes
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as exep:
            print(exep)

#### Do a SELECT to verify that the data have been inserted into third table

In [69]:
## Executing the select query to verify if the data was entered into the table

select_query = "SELECT first_name, last_name                   \
                FROM play_history                              \
                WHERE song_title='All Hands Against His Own'   ;"

try:
    rows = session.execute(select_query)
except Exception as exep:
    print(exep)

# Print the results
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [4]:
## Dropping all the tables

In [70]:
try:
    session.execute("DROP TABLE IF EXISTS music_library")
except Execption as exep:
    print(exep)
    
try:
    session.execute("DROP TABLE IF EXISTS artist_library")
except Execption as exep:
    print(exep)
    
try:
    session.execute("DROP TABLE IF EXISTS play_history")
except Execption as exep:
    print(exep)  

### Close the session and cluster connection¶

In [71]:
session.shutdown()
cluster.shutdown()